<a href="https://colab.research.google.com/github/mhuggi/computer-vision/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
!ls

3rdparty	CMakeLists.txt		data		       obj
backup		darknet			image_yolov3.sh        README.md
bad_label.list	darknet53.conv.74	image_yolov4.sh        results
bad.list	darknet53.conv.74.1	include		       scripts
build		darknet53.conv.74.2	json_mjpeg_streams.sh  src
build.ps1	DarknetConfig.cmake.in	LICENSE		       vcpkg.json
build.sh	darknet_images.py	Makefile	       video_yolov3.sh
cfg		darknet.py		net_cam_v3.sh	       video_yolov4.sh
cmake		darknet_video.py	net_cam_v4.sh


In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Mar 12 20:25:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive


Mounted at /content/gdrive


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14730, done.
remote: Total 14730 (delta 0), reused 0 (delta 0), pack-reused 14730
Receiving objects: 100% (14730/14730), 13.27 MiB | 25.70 MiB/s, done.
Resolving deltas: 100% (10020/10020), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
#!rm -d -r data/obj/

!printf "King\nKnight" > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

In [ ]:
!ls data/obj/images/

20210308_151007.jpg  20210308_151942.jpg  20210308_162319.txt
20210308_151007.txt  20210308_151942.txt  20210308_162321.jpg
20210308_151011.jpg  20210308_151946.jpg  20210308_162321.txt
20210308_151011.txt  20210308_151946.txt  20210308_162323.jpg
20210308_151045.jpg  20210308_151951.jpg  20210308_162325.jpg
20210308_151045.txt  20210308_151951.txt  20210308_162325.txt
20210308_151050.jpg  20210308_151959.jpg  20210308_162329.jpg
20210308_151050.txt  20210308_151959.txt  20210308_162329.txt
20210308_151057.jpg  20210308_152003.jpg  20210308_162331.jpg
20210308_151057.txt  20210308_152003.txt  20210308_162331.txt
20210308_151105.jpg  20210308_152006.jpg  20210308_162333.jpg
20210308_151105.txt  20210308_152006.txt  20210308_162333.txt
20210308_151113.jpg  20210308_152013.jpg  20210308_162335.jpg
20210308_151113.txt  20210308_152013.txt  20210308_162335.txt
20210308_151120.jpg  20210308_152018.jpg  20210308_162337.jpg
20210308_151120.txt  20210308_152018.txt  20210308_162337.txt
20210308

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-03-11 13:37:29--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.3’

darknet53.conv.74.3 100%[===================>] 154.96M  20.3MB/s    in 8.5s    

2021-03-11 13:37:38 (18.3 MB/s) - ‘darknet53.conv.74.3’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/images/20210308_151007.jpg  
  inflating: data/obj/images/20210308_151007.txt  
  inflating: data/obj/images/20210308_151011.jpg  
  inflating: data/obj/images/20210308_151011.txt  
  inflating: data/obj/images/20210308_151045.jpg  
  inflating: data/obj/images/20210308_151045.txt  
  inflating: data/obj/images/20210308_151050.jpg  
  inflating: data/obj/images/20210308_151050.txt  
  inflating: data/obj/images/20210308_151057.jpg  
  inflating: data/obj/images/20210308_151057.txt  
  inflating: data/obj/images/20210308_151105.jpg  
  inflating: data/obj/images/20210308_151105.txt  
  inflating: data/obj/images/20210308_151113.jpg  
  inflating: data/obj/images/20210308_151113.txt  
  inflating: data/obj/images/20210308_151120.jpg  
  inflating: data/obj/images/20210308_151120.txt  
  inflating: data/obj/images/20210308_151129.jpg  
  inflating: data/obj/images/20210308_151129.txt  
  inflating: data/obj/images/20210308_151142.

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            numbers = re.findall("[0-9.]+", line)
            if numbers:
                num = int(numbers[0]) - 15


                # Define coordinates
                text = "{} {} {} {} {}".format(num, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 data/obj/images/20210308_151142.txt
0 0.808201 0.830192 0.117725 0.226521
0 data/obj/images/20210308_151142.txt
1 0.163360 0.866237 0.109788 0.151786
0 data/obj/images/20210308_151142.txt
1 0.642196 0.656581 0.089947 0.129299
1 data/obj/images/20210308_162416.txt
0 0.203704 0.243221 0.080688 0.176918
2 data/obj/images/20210308_151155.txt
1 0.641204 0.659722 0.089947 0.137566
2 data/obj/images/20210308_151155.txt
1 0.162864 0.861607 0.104167 0.157077
3 data/obj/images/20210308_152018.txt
0 0.540013 0.703042 0.093915 0.205688
4 data/obj/images/20210308_162351.txt
0 0.727017 0.522817 0.102183 0.196429
5 data/obj/images/20210308_151951.txt
0 0.792163 0.723710 0.108796 0.213955
6 data/obj/images/20210308_162430.txt
1 0.372024 0.539683 0.086640 0.123016
6 data/obj/images/20210308_162430.txt
0 0.272817 0.508267 0.086640 0.196429
7 data/obj/images/20210308_151120.txt
1 0.642692 0.655093 0.090939 0.132275
7 data/obj/images/20210308_151120.txt
1 0.159226 0.855985 0.129960 0.155754
8 data/obj/i

In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/20210308_151214.jpg', 'data/obj/images/20210308_151050.jpg', 'data/obj/images/20210308_162310.jpg', 'data/obj/images/20210308_151706.jpg', 'data/obj/images/20210308_162350.jpg', 'data/obj/images/20210308_162256.jpg', 'data/obj/images/20210308_162424.jpg', 'data/obj/images/20210308_162254.jpg', 'data/obj/images/20210308_151155.jpg', 'data/obj/images/20210308_151511.jpg', 'data/obj/images/20210308_162434.jpg', 'data/obj/images/20210308_151113.jpg', 'data/obj/images/20210308_151349.jpg', 'data/obj/images/20210308_162414.jpg', 'data/obj/images/20210308_162430.jpg', 'data/obj/images/20210308_151045.jpg', 'data/obj/images/20210308_151337.jpg', 'data/obj/images/20210308_162249.jpg', 'data/obj/images/20210308_151738.jpg', 'data/obj/images/20210308_162446.jpg', 'data/obj/images/20210308_162407.jpg', 'data/obj/images/20210308_151152.jpg', 'data/obj/images/20210308_162348.jpg', 'data/obj/images/20210308_151433.jpg', 'data/obj/images/20210308_151951.jpg', 'data/obj/images/2021030

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Striimattava tuloste lyhennetty viimeiseen 5000 riviin.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.818738), count: 6, class_loss = 0.011443, iou_loss = 0.177055, total_loss = 0.188499 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.755640), count: 1, class_loss = 0.235231, iou_loss = 0.036789, total_loss = 0.272020 
 total_bbox = 79884, rewritten_bbox = 0.010015 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000367, iou_loss = 0.000000, total_loss = 0.000367 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.891399), count: 7, class_loss = 0.072117, iou_loss = 0.062286, total_loss = 0.134403 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000010, iou_loss = 0.000000, total_loss = 0.000010 
 total_bbox = 79891, rewritten_bbox = 0.010014 % 
v3

In [ ]:
while True:pass


KeyboardInterrupt: ignored